In [1]:
import cv2
import dlib
import numpy as np
from imutils import face_utils

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('C:/Users/s_csmscox/jupyterSave/eye_blink/shape_predictor_68_face_landmarks.dat')

img_size = (32, 32)

# def mse(imageA, imageB):
#     err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
#     err /= float(imageA.shape[0] * imageB.shape[1])
#     return err

def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
    
    w = (x2 - x1) * 1.2
    h = w * img_size[1] / img_size[0]
    
    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)

    eye_img = img[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]
    
    return eye_img, eye_rect

In [9]:
# 영상을 이용
import time

cap = cv2.VideoCapture('C:/Users/s_csmscox/jupyterSave/eye_blink/lsh1.mp4')

ret, img = cap.read()
img = cv2.resize(img, dsize=(0, 0), fx=0.5, fy=0.5)

total_start = time.time()
frame_cnt = 0

eye_alert_count = 0

nose_sum = 0
nose_cnt = 0
nose_mean = 0
nose_alert_cnt = 0

mouse_sum = 0
mouse_cnt = 0
mouse_mean = 0
mouse_alert_cnt = 0

while True:
    start = time.time()  # 시작 시간 저장
    
    ret, compare_img = cap.read()

    if not ret:
        break
        
    if frame_cnt % 2 != 0:
        frame_cnt += 1
        continue
    else:
        frame_cnt += 1

    compare_img = cv2.resize(compare_img, dsize=(0, 0), fx=0.5, fy=0.5)

    faces = detector(compare_img)
    
    for face in faces:
        shapes = predictor(compare_img, face)
        shapes = face_utils.shape_to_np(shapes)

        eye_img_l, eye_rect_l = crop_eye(compare_img, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(compare_img, eye_points=shapes[42:48])

        eye_img_l = cv2.resize(eye_img_l, dsize=img_size)
        eye_img_r = cv2.resize(eye_img_r, dsize=img_size)

        # 왼쪽 눈
        eye_input_l = eye_img_l.copy().reshape((1, img_size[1], img_size[0], 3)).astype(np.float32)
        eye_input_l = eye_input_l/255

#         with tf.device('/cpu:0'):
#             pred_l = model.call(tf.convert_to_tensor(eye_input_l), training = False)
        
#         pred_l = model.call(tf.convert_to_tensor(eye_input_l), training = False)
#         pred_l = np.argmax(pred_l)

        # 오른쪽 눈
        eye_input_r = eye_img_r.copy().reshape((1, img_size[1], img_size[0], 3)).astype(np.float32)
        eye_input_r = eye_input_r/255

#         with tf.device('/cpu:0'):
#             pred_r = model.call(tf.convert_to_tensor(eye_input_r), training = False)
            
#         pred_r = model.call(tf.convert_to_tensor(eye_input_r), training = False)
#         pred_r = np.argmax(pred_r)

        # 눈에 직사각형 그리기
        cv2.rectangle(compare_img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255,255,255), thickness=2)
        cv2.rectangle(compare_img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255,255,255), thickness=2)

#         cv2.putText(compare_img, str(pred_l), tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
#         cv2.putText(compare_img, str(pred_r), tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

        # 입에 직사각형 그리기
        cv2.rectangle(compare_img, pt1=(shapes[48][0],shapes[51][1]), pt2=(shapes[54][0],shapes[57][1]), color=(0,255,0), thickness=2)
        
        # 두 눈 다 감은 경우 졸음으로 예측
#         if pred_l == 0 and pred_r == 0:
#             eye_alert_count += 1
#         else:
#             eye_alert_count = 0

#         # n_count가 10 초과하면 경고 메세지
#         if eye_alert_count > 10:
#             cv2.putText(compare_img, "Wake up!(eye_blink)", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2) #BGR
#             print("Wake up!(movement)")

#         if nose_cnt == 0:
#             nose_sum += shapes[33][1]
#             nose_cnt += 1
#             nose_mean = nose_sum / nose_cnt
#         elif shapes[33][1] - nose_mean > 10:
#             nose_alert_cnt += 1
#         else:
#             nose_sum += shapes[33][1]
#             nose_cnt += 1
#             nose_mean = nose_sum / nose_cnt
#             nose_alert_cnt = 0
            
#         if nose_alert_cnt > 10:
#             cv2.putText(compare_img, "Wake up!(movement)", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
#             print("Wake up!(movement)")
            
#         # 입 특징점 기반으로 하품 감지
#         mouse_rate = (shapes[57][1] - shapes[51][1])/(shapes[54][0] - shapes[48][0])
        
#         if mouse_cnt == 0:
#             mouse_sum += mouse_rate
#             mouse_cnt += 1
#             mouse_mean = mouse_sum / mouse_cnt
#         elif mouse_rate - mouse_mean > 0.2:
#             mouse_alert_cnt += 1
#         else:
#             mouse_sum += mouse_rate
#             mouse_cnt += 1
#             mouse_mean = mouse_sum / mouse_cnt
#             mouse_alert_cnt = 0
        
#         if mouse_alert_cnt > 10:
#             cv2.putText(compare_img, "Wake up!(yawning)", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
#             print("Wake up!(yawning)")
            

    cv2.imshow('result', eye_img_r)
    print("time :", time.time() - start)

    # ESC 누르면 영상 종료
    if cv2.waitKey(27) > 0:
        break
        
print("total_time :", time.time() - total_start)
print("frame_count: ",frame_cnt)
    
cap.release()
cv2.destroyAllWindows()

time : 0.02001810073852539
time : 0.011009693145751953
time : 0.011010169982910156
time : 0.013011693954467773
time : 0.012011051177978516
time : 0.012011051177978516
time : 0.012010812759399414
time : 0.011010169982910156
time : 0.01101064682006836
time : 0.011009454727172852
time : 0.01401209831237793
time : 0.011011123657226562
time : 0.01176762580871582
time : 0.012011289596557617
time : 0.012141227722167969
time : 0.011010169982910156
time : 0.014013528823852539
time : 0.012010812759399414
time : 0.012011051177978516
time : 0.012009859085083008
time : 0.012011051177978516
time : 0.012011289596557617
time : 0.011010408401489258
time : 0.011009931564331055
time : 0.011010408401489258
time : 0.011009931564331055
time : 0.011009693145751953
time : 0.015013933181762695
time : 0.011010169982910156
time : 0.012011289596557617
time : 0.01101064682006836
time : 0.014012575149536133
time : 0.01101064682006836
time : 0.011009931564331055
time : 0.012011289596557617
time : 0.01101016998291015